# Hyperparameters and lags search: backtesting vs one-step-ahead

Hyperparameter and lag tuning involves systematically testing different values or combinations of hyperparameters (and/or lags) to find the optimal configuration that gives the best performance. The **skforecast** library provides two different methods to evaluate each candidate configuration:

+ Backtesting: In this method, the model predicts several steps ahead in each iteration, using the same forecast horizon and retraining frequency strategy that would be used if the model were deployed. This simulates a real forecasting scenario where the model is retrained and updated over time.

+ One-Step Ahead: Evaluates the model using only one-step-ahead predictions. This method is faster because it requires fewer iterations, but it only tests the model's performance in the immediate next time step.

Each method uses a different evaluation strategy, so they may produce different results. However, in the long run, both methods are expected to converge to similar selections of optimal hyperparameters. The one-step-ahead method is much faster than backtesting because it requires fewer iterations, but it only tests the model's performance in the immediate next time step. It is recommended to backtest the final model for a more accurate multi-step performance estimate.

This document compares the results of each approach when applied to different forecasters and data sets.

!!!!!!Mostrar plot con los resultados!!!

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'/home/ubuntu/varios/skforecast'

In [2]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
from time import time
from copy import copy
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from skforecast.datasets import fetch_dataset
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import bayesian_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.ForecasterAutoregMultiVariate import ForecasterAutoregMultiVariate
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries import grid_search_forecaster_multiseries
from skforecast.model_selection_multiseries import bayesian_search_forecaster_multiseries

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Functions to compare results using backtesting and one step ahead
# ==============================================================================
def grid_search_benchmark(
    data,
    forecaster_to_benchmark,
    lags_grid,
    param_grid,
    end_train,
    end_validation,
    target,
    exog_features,
    steps,
    metric
):
    """
    Compare results of grid search using backtesting and one-step-ahead.
    """
    
    # Method: backtesting
    forecaster = copy(forecaster_to_benchmark)
    start  = time()
    results_1 = grid_search_forecaster(
                forecaster         = forecaster,
                y                  = data.loc[:end_validation, target],
                exog               = data.loc[:end_validation, exog_features] if exog_features else None,
                param_grid         = param_grid,
                lags_grid          = lags_grid,
                steps              = steps,
                refit              = False,
                metric             = metric,
                initial_train_size = len(data.loc[:end_train]),
                method             = 'backtesting',
                fixed_train_size   = False,
                return_best        = True,
                n_jobs             = 'auto',
                verbose            = False,
                show_progress      = True
            )
    end = time()
    time_1 = end - start
    metric_1, predictions_1 = backtesting_forecaster(
        forecaster          = forecaster,
        y                   = data.loc[:, target],
        exog                = data.loc[:, exog_features] if exog_features else None,
        initial_train_size  = len(data.loc[:end_validation]),
        steps               = steps,
        metric              = metric,
        verbose             = False,
        show_progress       = False
    )

    # Method: one step ahead
    forecaster = copy(forecaster_to_benchmark)
    start  = time()
    results_2 = grid_search_forecaster(
                forecaster         = forecaster,
                y                  = data.loc[:end_validation, target],
                exog               = data.loc[:end_validation, exog_features] if exog_features else None,
                steps              = steps,
                param_grid         = param_grid,
                lags_grid          = lags_grid,
                metric             = metric,
                initial_train_size = len(data.loc[:end_train]),
                method             = 'one_step_ahead',
                return_best        = True,
                verbose            = False,
                show_progress      = True
            )
    end = time()
    time_2 = end - start
    metric_2, predictions_2 = backtesting_forecaster(
        forecaster          = forecaster,
        y                   = data.loc[:, target],
        exog                = data.loc[:, exog_features] if exog_features else None,
        initial_train_size  = len(data.loc[:end_validation]),
        steps               = steps,
        metric              = metric,
        verbose             = False,
        show_progress       = False
    )

    print("Benchmark results")
    print("-----------------")
    print('Execution time backtesting   :', time_1)
    print('Execution time one step ahead:', time_2)
    print(f"Same lags  : {np.array_equal(results_1.loc[0, 'lags'], results_2.loc[0, 'lags'])}")
    print(f"Same params: {results_1.loc[0, 'params'] == results_2.loc[0, 'params']}")
    print("")
    print("Method: backtesting")
    print(f"    lags   : {results_1.loc[0, 'lags']}")
    print(f"    params : {results_1.loc[0, 'params']}")
    print(f"    metrics: {metric_1.loc[0, metric]}")
    print("")
    print("Method: one step ahead")
    print(f"    lags   : {results_2.loc[0, 'lags']}")
    print(f"    params : {results_2.loc[0, 'params']}")
    print(f"    metrics: {metric_2.loc[0, metric]}")
    
    return time_1, time_2, metric_1.loc[0, metric], metric_2.loc[0, metric]


def bayesian_search_benchmark(
    data,
    forecaster_to_benchmark,
    search_space,
    end_train,
    end_validation,
    target,
    exog_features,
    steps,
    metric
):
    """
    Compare results of bayesian search using backtesting and one-step-ahead.
    """

    # Method: backtesting
    forecaster = copy(forecaster_to_benchmark)
    start  = time()
    results_1, _ = bayesian_search_forecaster(
                        forecaster         = forecaster,
                        y                  = data.loc[:end_validation, target],
                        exog               = data.loc[:end_validation, exog_features] if exog_features else None,
                        search_space       = search_space,
                        steps              = steps,
                        refit              = False,
                        metric             = metric,
                        initial_train_size = len(data.loc[:end_train]),
                        method             = 'backtesting',
                        fixed_train_size   = False,
                        n_trials           = 20,
                        random_state       = 123,
                        return_best        = True,
                        n_jobs             = 'auto',
                        verbose            = False,
                        show_progress      = True
                    )
    end = time()
    time_1 = end - start
    metric_1, predictions_1 = backtesting_forecaster(
        forecaster          = forecaster,
        y                   = data.loc[:, target],
        exog                = data.loc[:, exog_features] if exog_features else None,
        initial_train_size  = len(data.loc[:end_validation]),
        steps               = steps,
        metric              = metric,
        verbose             = False,
        show_progress       = False
    )

    # Method: one step ahead
    forecaster = copy(forecaster_to_benchmark)
    start  = time()
    results_2, _ = bayesian_search_forecaster(
                        forecaster         = forecaster,
                        y                  = data.loc[:end_validation, target],
                        exog               = data.loc[:end_validation, exog_features] if exog_features else None,
                        steps              = steps,
                        search_space       = search_space,
                        metric             = metric,
                        initial_train_size = len(data.loc[:end_train]),
                        method             = 'one_step_ahead',
                        n_trials           = 20,
                        random_state       = 123,
                        return_best        = True,
                        verbose            = False,
                        show_progress      = True
                    )
    end = time()
    time_2 = end - start
    metric_2, predictions_2 = backtesting_forecaster(
        forecaster          = forecaster,
        y                   = data.loc[:, target],
        exog                = data.loc[:, exog_features] if exog_features else None,
        initial_train_size  = len(data.loc[:end_validation]),
        steps               = steps,
        metric              = metric,
        verbose             = False,
        show_progress       = False
    )

    print("Benchmark results")
    print("-----------------")
    print('Execution time backtesting   :', time_1)
    print('Execution time one step ahead:', time_2)
    print(f"Same lags  : {np.array_equal(results_1.loc[0, 'lags'], results_2.loc[0, 'lags'])}")
    print(f"Same params: {results_1.loc[0, 'params'] == results_2.loc[0, 'params']}")
    print("")
    print("Method: backtesting")
    print(f"    lags   : {results_1.loc[0, 'lags']}")
    print(f"    params : {results_1.loc[0, 'params']}")
    print(f"    metrics: {metric_1.loc[0, metric]}")
    print("")
    print("Method: one step ahead")
    print(f"    lags   : {results_2.loc[0, 'lags']}")
    print(f"    params : {results_2.loc[0, 'params']}")
    print(f"    metrics: {metric_2.loc[0, metric]}")

    return time_1, time_2, metric_1.loc[0, metric], metric_2.loc[0, metric]

In [4]:
# Table to store results
# ==============================================================================
results_grid_search = pd.DataFrame(
    columns=[
        "forecaster",
        "time_search_backtesting",
        "time_search_one_step",
        "metric_backtesting",
        "metric_one_step",
    ],
    index=[
        "h2o",
        "bike_sharing_extended_features",
        "website_visits",
        "vic_electricity",
    ]
)
results_bayesian_search = results_grid_search.copy()

## Forecaster Autoreg

### Benchmark: H2O dataset

In [5]:
# Data
# ==============================================================================
data = fetch_dataset(
    name="h2o",
    raw=True,
    kwargs_read_csv={"names": ["y", "datetime"], "header": 0},
    verbose=False,
)
data["datetime"] = pd.to_datetime(data["datetime"], format="%Y-%m-%d")
data = data.set_index("datetime")
data = data.asfreq("MS")
data = data[["y"]]
data = data.sort_index()
end_train = "2001-01-01 23:59:00"
end_validation = "2006-01-01 23:59:00"

In [6]:
# Grid search hyperparameters and lags
# ==============================================================================
forecaster = ForecasterAutoreg(
                regressor = LGBMRegressor(random_state=123, verbose=-1),
                lags      = 10
            )
lags_grid = [3, 10, [1, 2, 3, 20]]
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [2, 3, 5]
}

time_1, time_2, metric_1, metric_2 = grid_search_benchmark(
    data                    = data,
    forecaster_to_benchmark = forecaster,
    lags_grid               = lags_grid,
    param_grid              = param_grid,
    end_train               = end_train,
    end_validation          = end_validation,
    target                  = 'y',
    exog_features           = None,
    steps                   = 12,
    metric                  = 'mean_absolute_error'
)
results_grid_search.loc["h2o", :] = [
    type(forecaster).__name__,
    time_1,
    time_2,
    metric_1,
    metric_2,
]

lags grid:   0%|          | 0/3 [00:00<?, ?it/s]

params grid:   0%|          | 0/6 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3 20] 
  Parameters: {'max_depth': 2, 'n_estimators': 100}
  Backtesting metric: 0.18240283368659635



lags grid:   0%|          | 0/3 [00:00<?, ?it/s]

params grid:   0%|          | 0/6 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10] 
  Parameters: {'max_depth': 2, 'n_estimators': 100}
  Backtesting metric: 0.16147435680094147

Benchmark results
-----------------
Execution time backtesting   : 1.0717644691467285
Execution time one step ahead: 0.21580886840820312
Same lags  : False
Same params: True

Method: backtesting
    lags   : [ 1  2  3 20]
    params : {'max_depth': 2, 'n_estimators': 100}
    metrics: 0.22615930424513603

Method: one step ahead
    lags   : [ 1  2  3  4  5  6  7  8  9 10]
    params : {'max_depth': 2, 'n_estimators': 100}
    metrics: 0.13519201944243026


### Benchmark: Bike sharing dataset

In [7]:
# Data
# ==============================================================================
data = fetch_dataset('bike_sharing_extended_features', verbose=False)
end_train = '2012-03-31 23:59:00'
end_validation = '2012-08-31 23:59:00'
data_train = data.loc[: end_train, :]
data_val   = data.loc[end_train:end_validation, :]
data_test  = data.loc[end_validation:, :]
exog_features = [
    'month_sin', 
    'month_cos',
    'week_of_year_sin',
    'week_of_year_cos',
    'week_day_sin',
    'week_day_cos',
    'hour_day_sin',
    'hour_day_cos',
    'sunrise_hour_sin',
    'sunrise_hour_cos',
    'sunset_hour_sin',
    'sunset_hour_cos',
    'holiday_previous_day',
    'holiday_next_day',
    'temp_roll_mean_1_day',
    'temp_roll_mean_7_day',
    'temp_roll_max_1_day',
    'temp_roll_min_1_day',
    'temp_roll_max_7_day',
    'temp_roll_min_7_day',
    'temp',
    'holiday'
]

In [8]:
# Grid search hyperparameters and lags
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = LGBMRegressor(random_state=123, verbose=-1),
                 lags      = 10
             )
lags_grid = [48, 72, (1, 2, 3, 23, 24, 25, 167, 168, 169)]
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [2, 3, 5],
    'learning_rate': [0.01, 0.1, 0.5]
}

grid_search_benchmark(
    data                    = data,
    forecaster_to_benchmark = forecaster,
    lags_grid               = lags_grid,
    param_grid              = param_grid,
    end_train               = end_train,
    end_validation          = end_validation,
    target                  = 'users',
    exog_features           = exog_features,
    steps                   = 36,
    metric                  = 'mean_absolute_error'
)

results_grid_search.loc["bike_sharing_extended_features", :] = [
    type(forecaster).__name__,
    time_1,
    time_2,
    metric_1,
    metric_2,
]

lags grid:   0%|          | 0/3 [00:00<?, ?it/s]

params grid:   0%|          | 0/27 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [136]:
# Bayesian search hyperparameters and lags
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = LGBMRegressor(random_state=123, verbose=-1),
                 lags      = 10
             )
lags_grid = [48, 72, [1, 2, 3, 23, 24, 25, 167, 168, 169]]
def search_space(trial):
    search_space  = {
        'n_estimators'    : trial.suggest_int('n_estimators', 400, 1200, step=100),
        'max_depth'       : trial.suggest_int('max_depth', 3, 10, step=1),
        'learning_rate'   : trial.suggest_float('learning_rate', 0.01, 1),
        'subsample'       : trial.suggest_float('subsample', 0.1, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
        'gamma'           : trial.suggest_float('gamma', 0, 1),
        'reg_alpha'       : trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda'      : trial.suggest_float('reg_lambda', 0, 1),
        'lags'            : trial.suggest_categorical('lags', lags_grid)
    } 
    return search_space

bayesian_search_benchmark(
    data                    = data,
    forecaster_to_benchmark = forecaster,
    search_space            = search_space,
    end_train               = end_train,
    end_validation          = end_validation,
    target                  = 'users',
    exog_features           = exog_features,
    steps                   = 36,
    metric                  = 'mean_absolute_error'
)
results_bayesian_search.loc["bike_sharing_extended_features", :] = [
    type(forecaster).__name__,
    time_1,
    time_2,
    metric_1,
    metric_2,
]

  0%|          | 0/20 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [  1   2   3  23  24  25 167 168 169] 
  Parameters: {'n_estimators': 1100, 'max_depth': 9, 'learning_rate': 0.010416860203198298, 'subsample': 0.2755690967419942, 'colsample_bytree': 0.6560647401138757, 'gamma': 0.10155013994756559, 'reg_alpha': 0.8864717945956972, 'reg_lambda': 0.014015836934027859}
  Backtesting metric: 64.00591134157926



/home/ubuntu/varios/skforecast/skforecast/model_selection/model_selection.py:2177: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


  0%|          | 0/20 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [  1   2   3  23  24  25 167 168 169] 
  Parameters: {'n_estimators': 1200, 'max_depth': 10, 'learning_rate': 0.022664761670265286, 'subsample': 0.11400878440779749, 'colsample_bytree': 0.9845292840882569, 'gamma': 0.004802254183477721, 'reg_alpha': 0.05556023765455381, 'reg_lambda': 0.22785658420818355}
  Backtesting metric: 43.1076403895807

Benchmark results
-----------------
Execution time backtesting   : 96.7142424583435
Execution time one step ahead: 32.7284996509552
Same lags  : True
Same params: False

Method: backtesting
    lags   : [  1   2   3  23  24  25 167 168 169]
    params : {'n_estimators': 1100, 'max_depth': 9, 'learning_rate': 0.010416860203198298, 'subsample': 0.2755690967419942, 'colsample_bytree': 0.6560647401138757, 'gamma': 0.10155013994756559, 'reg_alpha': 0.8864717945956972, 'reg_lambda': 0.014015836934027859}
    metrics: 54.63516407405189

Method: one step ahea

### Benchmark: website visits dataset

In [14]:
# Data
# ==============================================================================
data = fetch_dataset(name="website_visits", raw=True, verbose=False)
data['date'] = pd.to_datetime(data['date'], format='%d/%m/%y')
data = data.set_index('date')
data = data.asfreq('1D')
data = data.sort_index()
data['month'] = data.index.month
data['month_day'] = data.index.day
data['week_day'] = data.index.day_of_week
one_hot_encoder = make_column_transformer(
    (
        OneHotEncoder(sparse_output=False, drop='if_binary'),
        ['month', 'week_day', 'month_day'],
    ),
    remainder="passthrough",
    verbose_feature_names_out=False,
).set_output(transform="pandas")
data = one_hot_encoder.fit_transform(data)
end_train = '2021-03-30 23:59:00'
end_validation = '2021-06-30 23:59:00'
data_train = data.loc[: end_train, :]
data_val   = data.loc[end_train:end_validation, :]
data_test  = data.loc[end_validation:, :]
exog_features = [col for col in data.columns if col.startswith(('month_', 'week_day_', 'month_day_'))]

In [15]:
# Grid search hyperparameters and lags
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor     = Ridge(random_state=123),
                 transformer_y = StandardScaler(),
                 lags          = 10
             )
lags_grid = [7, 14, 21, [7, 14, 21]]
param_grid = {'alpha': np.logspace(-3, 3, 10)}

grid_search_benchmark(
    data                    = data,
    forecaster_to_benchmark = forecaster,
    lags_grid               = lags_grid,
    param_grid              = param_grid,
    end_train               = end_train,
    end_validation          = end_validation,
    target                  = 'users',
    exog_features           = exog_features,
    steps                   = 7,
    metric                  = 'mean_absolute_error'
)
results_grid_search.loc["website_visits", :] = [
    type(forecaster).__name__,
    time_1,
    time_2,
    metric_1,
    metric_2,
]

lags grid:   0%|          | 0/4 [00:00<?, ?it/s]

params grid:   0%|          | 0/10 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14] 
  Parameters: {'alpha': 10.0}
  Backtesting metric: 92723.37385771735



/home/ubuntu/varios/skforecast/skforecast/model_selection/model_selection.py:1435: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


lags grid:   0%|          | 0/4 [00:00<?, ?it/s]

params grid:   0%|          | 0/10 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14] 
  Parameters: {'alpha': 0.46415888336127775}
  Backtesting metric: 0.03135946751580578

Benchmark results
-----------------
Execution time backtesting   : 3.9623405933380127
Execution time one step ahead: 0.4341268539428711
Same lags  : True
Same params: False

Method: backtesting
    lags   : [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14]
    params : {'alpha': 10.0}
    metrics: 165.59443777438304

Method: one step ahead
    lags   : [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14]
    params : {'alpha': 0.46415888336127775}
    metrics: 170.6888738025913


In [16]:
# Bayesian search hyperparameters and lags
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor     = Ridge(random_state=123),
                 transformer_y = StandardScaler(),
                 lags          = 10
             )
lags_grid = [7, 14, 21, [7, 14, 21]]
def search_space(trial):
    search_space  = {
        'alpha' : trial.suggest_float('alpha', 0.001, 1000, log=True),
        'lags'  : trial.suggest_categorical('lags', lags_grid)
    } 
    return search_space

bayesian_search_benchmark(
    data                    = data,
    forecaster_to_benchmark = forecaster,
    search_space            = search_space,
    end_train               = end_train,
    end_validation          = end_validation,
    target                  = 'users',
    exog_features           = exog_features,
    steps                   = 7,
    metric                  = 'mean_absolute_error'
)
results_bayesian_search.loc["website_visits", :] = [
    type(forecaster).__name__,
    time_1,
    time_2,
    metric_1,
    metric_2,
]

  0%|          | 0/20 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14] 
  Parameters: {'alpha': 1.2524531789882662}
  Backtesting metric: 224.33854613853265



/home/ubuntu/varios/skforecast/skforecast/model_selection/model_selection.py:2177: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


  0%|          | 0/20 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14] 
  Parameters: {'alpha': 2.083580929967405}
  Backtesting metric: 0.13822004658108406

Benchmark results
-----------------
Execution time backtesting   : 2.0646800994873047
Execution time one step ahead: 0.3663356304168701
Same lags  : True
Same params: False

Method: backtesting
    lags   : [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14]
    params : {'alpha': 1.2524531789882662}
    metrics: 166.84400582475766

Method: one step ahead
    lags   : [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14]
    params : {'alpha': 2.083580929967405}
    metrics: 164.24239262308248


### Benchmark: electricity consumption dataset

In [17]:
# Data
# ==============================================================================
data = fetch_dataset(name='vic_electricity', raw=False, verbose=False)
data = data.drop(columns="Date")
data = (
    data
    .resample(rule="H", closed="left", label="right")
    .agg({
        "Demand": "mean",
        "Temperature": "mean",
        "Holiday": "mean",
    })
)
data = data.loc['2012-01-01 00:00:00': '2014-12-30 23:00:00'].copy()
end_train = '2013-12-31 23:59:00'
end_validation = '2014-11-30 23:59:00'
data_train = data.loc[: end_train, :].copy()
data_val   = data.loc[end_train:end_validation, :].copy()
data_test  = data.loc[end_validation:, :].copy()
exog_features = ['Temperature', 'Holiday']

/tmp/ipykernel_7071/1461443256.py:7: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample(rule="H", closed="left", label="right")


In [18]:
# Grid search hyperparameters and lags
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = LGBMRegressor(random_state=123, verbose=-1),
                 lags      = 10
             )
lags_grid = [48, 72, (1, 2, 3, 23, 24, 25, 167, 168, 169)]
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [2, 3, 5],
    'learning_rate': [0.01, 0.1, 0.5]
}

grid_search_benchmark(
    data                    = data,
    forecaster_to_benchmark = forecaster,
    lags_grid               = lags_grid,
    param_grid              = param_grid,
    end_train               = end_train,
    end_validation          = end_validation,
    target                  = 'Demand',
    exog_features           = exog_features,
    steps                   = 36,
    metric                  = 'mean_absolute_error'
)
results_grid_search.loc["vic_electricity", :] = [
    type(forecaster).__name__,
    time_1,
    time_2,
    metric_1,
    metric_2,
]

lags grid:   0%|          | 0/3 [00:00<?, ?it/s]

params grid:   0%|          | 0/27 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [  1   2   3  23  24  25 167 168 169] 
  Parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}
  Backtesting metric: 94957.10876856494



/home/ubuntu/varios/skforecast/skforecast/model_selection/model_selection.py:1435: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


lags grid:   0%|          | 0/3 [00:00<?, ?it/s]

params grid:   0%|          | 0/27 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [  1   2   3  23  24  25 167 168 169] 
  Parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}
  Backtesting metric: 9478.226436015197

Benchmark results
-----------------
Execution time backtesting   : 490.7595374584198
Execution time one step ahead: 18.60087752342224
Same lags  : True
Same params: True

Method: backtesting
    lags   : [  1   2   3  23  24  25 167 168 169]
    params : {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}
    metrics: 245.7862163569872

Method: one step ahead
    lags   : [  1   2   3  23  24  25 167 168 169]
    params : {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}
    metrics: 245.7862163569872


In [19]:
# Bayesian search hyperparameters and lags
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = LGBMRegressor(random_state=123, verbose=-1),
                 lags      = 10
             )
lags_grid = [48, 72, [1, 2, 3, 23, 24, 25, 167, 168, 169]]
def search_space(trial):
    search_space  = {
        'n_estimators'    : trial.suggest_int('n_estimators', 400, 1200, step=100),
        'max_depth'       : trial.suggest_int('max_depth', 3, 10, step=1),
        'learning_rate'   : trial.suggest_float('learning_rate', 0.01, 1),
        'subsample'       : trial.suggest_float('subsample', 0.1, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
        'gamma'           : trial.suggest_float('gamma', 0, 1),
        'reg_alpha'       : trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda'      : trial.suggest_float('reg_lambda', 0, 1),
        'lags'            : trial.suggest_categorical('lags', lags_grid)
    } 
    return search_space

bayesian_search_benchmark(
    data                    = data,
    forecaster_to_benchmark = forecaster,
    search_space            = search_space,
    end_train               = end_train,
    end_validation          = end_validation,
    target                  = 'Demand',
    exog_features           = exog_features,
    steps                   = 36,
    metric                  = 'mean_absolute_error'
)
results_bayesian_search.loc["vic_electricity", :] = [
    type(forecaster).__name__,
    time_1,
    time_2,
    metric_1,
    metric_2,
]

  0%|          | 0/20 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [  1   2   3  23  24  25 167 168 169] 
  Parameters: {'n_estimators': 1000, 'max_depth': 6, 'learning_rate': 0.06908111764347266, 'subsample': 0.4582398297973883, 'colsample_bytree': 0.7641958651588321, 'gamma': 0.18249173045349998, 'reg_alpha': 0.17545175614749253, 'reg_lambda': 0.5315513738418384}
  Backtesting metric: 191.8626391693285



/home/ubuntu/varios/skforecast/skforecast/model_selection/model_selection.py:2177: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


  0%|          | 0/20 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [  1   2   3  23  24  25 167 168 169] 
  Parameters: {'n_estimators': 1200, 'max_depth': 10, 'learning_rate': 0.05679576034204338, 'subsample': 0.10725502906361839, 'colsample_bytree': 0.9892976403836156, 'gamma': 0.006453374219829056, 'reg_alpha': 0.0038381442337546146, 'reg_lambda': 0.24053684218482807}
  Backtesting metric: 55.35922291628681

Benchmark results
-----------------
Execution time backtesting   : 158.8924789428711
Execution time one step ahead: 28.4509916305542
Same lags  : True
Same params: False

Method: backtesting
    lags   : [  1   2   3  23  24  25 167 168 169]
    params : {'n_estimators': 1000, 'max_depth': 6, 'learning_rate': 0.06908111764347266, 'subsample': 0.4582398297973883, 'colsample_bytree': 0.7641958651588321, 'gamma': 0.18249173045349998, 'reg_alpha': 0.17545175614749253, 'reg_lambda': 0.5315513738418384}
    metrics: 231.47139120775364

Method: one step ah

## Forecaster AutoregDirect

In [20]:
# Data
# ==============================================================================
data = fetch_dataset('bike_sharing_extended_features', verbose=False)
end_train = '2012-03-31 23:59:00'
end_validation = '2012-08-31 23:59:00'
data_train = data.loc[: end_train, :]
data_val   = data.loc[end_train:end_validation, :]
data_test  = data.loc[end_validation:, :]
exog_features = [
    'month_sin', 
    'month_cos',
    'week_of_year_sin',
    'week_of_year_cos',
    'week_day_sin',
    'week_day_cos',
    'hour_day_sin',
    'hour_day_cos',
    'sunrise_hour_sin',
    'sunrise_hour_cos',
    'sunset_hour_sin',
    'sunset_hour_cos',
    'holiday_previous_day',
    'holiday_next_day',
    'temp_roll_mean_1_day',
    'temp_roll_mean_7_day',
    'temp_roll_max_1_day',
    'temp_roll_min_1_day',
    'temp_roll_max_7_day',
    'temp_roll_min_7_day',
    'temp',
    'holiday'
]

# Grid search hyperparameters and lags
# ==============================================================================
forecaster = ForecasterAutoregDirect(
                 regressor = Ridge(random_state=123),
                 steps     = 5,
                 lags      = 10
             )
lags_grid = [48, 72, [1, 2, 3, 23, 24, 25, 167, 168, 169]]
param_grid = {'alpha': np.logspace(-3, 3, 10)}

grid_search_benchmark(
    data                    = data,
    forecaster_to_benchmark = forecaster,
    lags_grid               = lags_grid,
    param_grid              = param_grid,
    end_train               = end_train,
    end_validation          = end_validation,
    target                  = 'users',
    exog_features           = exog_features,
    steps                   = 5,
    metric                  = 'mean_absolute_error'
)
results_grid_search.loc["bike_sharing_extended_features", :] = [
    type(forecaster).__name__,
    time_1,
    time_2,
    metric_1,
    metric_2,
]

lags grid:   0%|          | 0/3 [00:00<?, ?it/s]

params grid:   0%|          | 0/10 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72] 
  Parameters: {'alpha': 1000.0}
  Backtesting metric: 10439.128519885262



/home/ubuntu/varios/skforecast/skforecast/model_selection/model_selection.py:1435: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


lags grid:   0%|          | 0/3 [00:00<?, ?it/s]

params grid:   0%|          | 0/10 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48] 
  Parameters: {'alpha': 46.41588833612773}
  Backtesting metric: 43422.71251085582

Benchmark results
-----------------
Execution time backtesting   : 67.24604392051697
Execution time one step ahead: 4.050391912460327
Same lags  : False
Same params: False

Method: backtesting
    lags   : [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72]
    params : {'alpha': 1000.0}
    metrics: 67.37528369177734

Method: one step ahead
    lags   : [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48]

In [21]:
# Bayesian search hyperparameters and lags
# ==============================================================================
forecaster = ForecasterAutoregDirect(
                 regressor = Ridge(random_state=123),
                 steps     = 5,
                 lags      = 10
             )
lags_grid = [48, 72, [1, 2, 3, 23, 24, 25, 167, 168, 169]]
def search_space(trial):
    search_space  = {
        'alpha' : trial.suggest_float('alpha', 0.001, 1000, log=True),
        'lags'  : trial.suggest_categorical('lags', lags_grid)
    } 
    return search_space

bayesian_search_benchmark(
    data                    = data,
    forecaster_to_benchmark = forecaster,
    search_space            = search_space,
    end_train               = end_train,
    end_validation          = end_validation,
    target                  = 'users',
    exog_features           = exog_features,
    steps                   = 5,
    metric                  = 'mean_absolute_error'
)
results_bayesian_search.loc["bike_sharing_extended_features", :] = [
    type(forecaster).__name__,
    time_1,
    time_2,
    metric_1,
    metric_2,
]

  0%|          | 0/20 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72] 
  Parameters: {'alpha': 925.1006260111275}
  Backtesting metric: 70.99353356420917



/home/ubuntu/varios/skforecast/skforecast/model_selection/model_selection.py:2177: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


  0%|          | 0/20 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48] 
  Parameters: {'alpha': 6.403328042606579}
  Backtesting metric: 149.21303984528487

Benchmark results
-----------------
Execution time backtesting   : 45.40154194831848
Execution time one step ahead: 2.209826707839966
Same lags  : False
Same params: False

Method: backtesting
    lags   : [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72]
    params : {'alpha': 925.1006260111275}
    metrics: 67.37630319670156

Method: one step ahead
    lags   : [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 

In [137]:
results_grid_search

,forecaster,time_search_backtesting,time_search_one_step,metric_backtesting,metric_one_step
h2o,ForecasterAutoreg,1.05103,0.21661,0.226159,0.135192
bike_sharing_extended_features,ForecasterAutoreg,1.05103,0.21661,0.226159,0.135192
website_visits,NaN,NaN,NaN,NaN,NaN
vic_electricity,NaN,NaN,NaN,NaN,NaN


In [138]:
results_bayesian_search

,forecaster,time_search_backtesting,time_search_one_step,metric_backtesting,metric_one_step
h2o,NaN,NaN,NaN,NaN,NaN
bike_sharing_extended_features,ForecasterAutoreg,1.05103,0.21661,0.226159,0.135192
website_visits,NaN,NaN,NaN,NaN,NaN
vic_electricity,NaN,NaN,NaN,NaN,NaN


## ForecasterAutoregMultiseries

In [34]:
# Functions to compare results using backtesting and one step ahead
# ==============================================================================
def grid_search_multiseries_benchmark(
    data,
    forecaster_to_benchmark,
    lags_grid,
    param_grid,
    end_train,
    end_validation,
    levels,
    exog_features,
    steps,
    metric
):
    """
    Compare results of grid search using backtesting and one-step-ahead.
    """
    
    # Method: backtesting
    forecaster = copy(forecaster_to_benchmark)
    start  = time()
    results_1 = grid_search_forecaster_multiseries(
                forecaster         = forecaster,
                series             = data.loc[:end_validation, levels],
                levels             = levels,
                exog               = data.loc[:end_validation, exog_features] if exog_features else None,
                param_grid         = param_grid,
                lags_grid          = lags_grid,
                steps              = steps,
                refit              = False,
                metric             = metric,
                initial_train_size = len(data.loc[:end_train]),
                method             = 'backtesting',
                fixed_train_size   = False,
                return_best        = True,
                n_jobs             = 'auto',
                verbose            = False,
                show_progress      = True
            )
    end = time()
    time_1 = end - start
    metric_1, predictions_1 = backtesting_forecaster_multiseries(
        forecaster          = forecaster,
        series              = data.loc[:, levels],
        exog                = data.loc[:, exog_features] if exog_features else None,
        initial_train_size  = len(data.loc[:end_validation]),
        levels              = levels,
        steps               = steps,
        metric              = metric,
        verbose             = False,
        show_progress       = False
    )

    # Method: one step ahead
    forecaster = copy(forecaster_to_benchmark)
    start  = time()
    results_2 = grid_search_forecaster_multiseries(
                forecaster         = forecaster,
                series             = data.loc[:end_validation, levels],
                exog               = data.loc[:end_validation, exog_features] if exog_features else None,
                levels             = levels,
                param_grid         = param_grid,
                lags_grid          = lags_grid,
                steps              = steps,
                metric             = metric,
                initial_train_size = len(data.loc[:end_train]),
                method             = 'one_step_ahead',
                return_best        = True,
                verbose            = False,
                show_progress      = True
            )
    end = time()
    time_2 = end - start
    metric_2, predictions_2 = backtesting_forecaster_multiseries(
        forecaster          = forecaster,
        series              = data.loc[:, levels],
        exog                = data.loc[:, exog_features] if exog_features else None,
        levels              = levels,
        initial_train_size  = len(data.loc[:end_validation]),
        steps               = steps,
        metric              = metric,
        verbose             = False,
        show_progress       = False
    )

    print("Benchmark results")
    print("-----------------")
    print('Execution time backtesting   :', time_1)
    print('Execution time one step ahead:', time_2)
    print(f"Same lags  : {np.array_equal(results_1.loc[0, 'lags'], results_2.loc[0, 'lags'])}")
    print(f"Same params: {results_1.loc[0, 'params'] == results_2.loc[0, 'params']}")
    print("")
    print("Method: backtesting")
    print(f"    lags   : {results_1.loc[0, 'lags']}")
    print(f"    params : {results_1.loc[0, 'params']}")
    print(f"    metrics: {metric_1.loc[0, metric]}")
    print("")
    print("Method: one step ahead")
    print(f"    lags   : {results_2.loc[0, 'lags']}")
    print(f"    params : {results_2.loc[0, 'params']}")
    print(f"    metrics: {metric_2.loc[0, metric]}")
    
    return time_1, time_2, metric_1.loc[0, metric], metric_2.loc[0, metric]


def bayesian_search_multiseries_benchmark(
    data,
    forecaster_to_benchmark,
    search_space,
    end_train,
    end_validation,
    levels,
    #target,
    exog_features,
    steps,
    metric
):
    """
    Compare results of bayesian search using backtesting and one-step-ahead.
    """

    # Method: backtesting
    forecaster = copy(forecaster_to_benchmark)
    start  = time()
    results_1, _ = bayesian_search_forecaster_multiseries(
                        forecaster         = forecaster,
                        series             = data.loc[:end_validation, levels],
                        exog               = data.loc[:end_validation, exog_features] if exog_features else None,
                        levels             = levels,
                        search_space       = search_space,
                        steps              = steps,
                        refit              = False,
                        metric             = metric,
                        initial_train_size = len(data.loc[:end_train]),
                        method             = 'backtesting',
                        fixed_train_size   = False,
                        n_trials           = 20,
                        random_state       = 123,
                        return_best        = True,
                        n_jobs             = 'auto',
                        verbose            = False,
                        show_progress      = True
                    )
    end = time()
    time_1 = end - start
    metric_1, predictions_1 = backtesting_forecaster_multiseries(
        forecaster          = forecaster,
        series              = data.loc[:, levels],
        exog                = data.loc[:, exog_features] if exog_features else None,
        levels              = levels,
        initial_train_size  = len(data.loc[:end_validation]),
        steps               = steps,
        metric              = metric,
        verbose             = False,
        show_progress       = False
    )

    # Method: one step ahead
    forecaster = copy(forecaster_to_benchmark)
    start  = time()
    results_2, _ = bayesian_search_forecaster_multiseries(
                        forecaster         = forecaster,
                        series             = data.loc[:end_validation, levels],
                        exog               = data.loc[:end_validation, exog_features] if exog_features else None,
                        levels             = levels,
                        steps              = steps,
                        search_space       = search_space,
                        metric             = metric,
                        initial_train_size = len(data.loc[:end_train]),
                        method             = 'one_step_ahead',
                        n_trials           = 20,
                        random_state       = 123,
                        return_best        = True,
                        verbose            = False,
                        show_progress      = True
                    )
    end = time()
    time_2 = end - start
    metric_2, predictions_2 = backtesting_forecaster_multiseries(
        forecaster          = forecaster,
        series              = data.loc[:, levels],
        exog                = data.loc[:, exog_features] if exog_features else None,
        levels              = levels,
        initial_train_size  = len(data.loc[:end_validation]),
        steps               = steps,
        metric              = metric,
        verbose             = False,
        show_progress       = False
    )

    print("Benchmark results")
    print("-----------------")
    print('Execution time backtesting   :', time_1)
    print('Execution time one step ahead:', time_2)
    print(f"Same lags  : {np.array_equal(results_1.loc[0, 'lags'], results_2.loc[0, 'lags'])}")
    print(f"Same params: {results_1.loc[0, 'params'] == results_2.loc[0, 'params']}")
    print("")
    print("Method: backtesting")
    print(f"    lags   : {results_1.loc[0, 'lags']}")
    print(f"    params : {results_1.loc[0, 'params']}")
    print(f"    metrics: {metric_1.loc[0, metric]}")
    print("")
    print("Method: one step ahead")
    print(f"    lags   : {results_2.loc[0, 'lags']}")
    print(f"    params : {results_2.loc[0, 'params']}")
    print(f"    metrics: {metric_2.loc[0, metric]}")

    return time_1, time_2, metric_1.loc[0, metric], metric_2.loc[0, metric]

In [35]:
# Table to store results
# ==============================================================================
results_grid_search = pd.DataFrame(
    columns=[
        "forecaster",
        "time_search_backtesting",
        "time_search_one_step",
        "metric_backtesting",
        "metric_one_step",
    ],
    index=[
        "items_sales",
    ]
)
results_bayesian_search = results_grid_search.copy()

In [36]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from skforecast.model_selection_multiseries.model_selection_multiseries import _calculate_metrics_multiseries_one_step_ahead
from skforecast.model_selection_multiseries.model_selection_multiseries import _calculate_metrics_multiseries
from skforecast.model_selection.model_selection import _create_backtesting_folds
from skforecast.metrics import add_y_train_argument
from skforecast.metrics import mean_absolute_scaled_error

In [37]:
# Data download
# ==============================================================================
data = fetch_dataset(name="items_sales")
end_train = '2014-05-15 23:59:00'
end_validation = '2014-07-15 23:59:00'
data_train = data.loc[:end_train, :].copy()
data_test  = data.loc[end_train:, :].copy()
levels = ['item_1', 'item_2', 'item_3']
exog_features = None 

items_sales
-----------
Simulated time series for the sales of 3 different items.
Simulated data.
Shape of the dataset: (1097, 3)


In [38]:
# Grid search hyperparameters and lags
# ==============================================================================
forecaster = ForecasterAutoregMultiSeries(
                 regressor          = LGBMRegressor(random_state=123, verbose=-1),
                 lags               = 24,
                 encoding           = "ordinal",
                 transformer_series = None,
                 transformer_exog   = None,
                 weight_func        = None,
                 series_weights     = None,
                 differentiation    = None,
                 dropna_from_series = False,
                 fit_kwargs         = None,
                 forecaster_id      = None
             )

lags_grid ={
    '24 lags': 24,
    '48 lags': 48
}

param_grid = {
    'n_estimators': [10, 20],
    'max_depth': [3, 7]
}

grid_search_multiseries_benchmark(
    data                    = data,
    forecaster_to_benchmark = forecaster,
    lags_grid               = lags_grid,
    param_grid              = param_grid,
    end_train               = end_train,
    end_validation          = end_validation,
    levels                  = levels,
    exog_features           = exog_features,
    steps                   = 36,
    metric                  = 'mean_absolute_error'
)
results_grid_search.loc["items_sales", :] = [
    type(forecaster).__name__,
    time_1,
    time_2,
    metric_1,
    metric_2,
]

lags grid:   0%|          | 0/2 [00:00<?, ?it/s]

params grid:   0%|          | 0/4 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24] 
  Parameters: {'max_depth': 7, 'n_estimators': 20}
  Backtesting metric: 2.281107158900761
  Levels: ['item_1', 'item_2', 'item_3']



lags grid:   0%|          | 0/2 [00:00<?, ?it/s]

params grid:   0%|          | 0/4 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48] 
  Parameters: {'max_depth': 3, 'n_estimators': 20}
  Backtesting metric: 1.7788941749503342
  Levels: ['item_1', 'item_2', 'item_3']

Benchmark results
-----------------
Execution time backtesting   : 1.6039478778839111
Execution time one step ahead: 0.7105159759521484
Same lags  : False
Same params: False

Method: backtesting
    lags   : [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]
    params : {'max_depth': 7, 'n_estimators': 20}
    metrics: 1.43839579684015

Method: one step ahead
    lags   : [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48]
    params : {'max_depth': 3, 'n_estimators': 20}
    metrics: 1.6408350320

In [39]:
# Bayesian search hyperparameters and lags
# ==============================================================================
forecaster = ForecasterAutoregMultiSeries(
                 regressor          = LGBMRegressor(random_state=123, verbose=-1),
                 lags               = 24,
                 encoding           = "ordinal",
                 transformer_series = None,
                 transformer_exog   = None,
                 weight_func        = None,
                 series_weights     = None,
                 differentiation    = None,
                 dropna_from_series = False,
                 fit_kwargs         = None,
                 forecaster_id      = None
             )

lags_grid = [48, 72]
def search_space(trial):
    search_space  = {
        'n_estimators'    : trial.suggest_int('n_estimators', 10, 50),
        'max_depth'       : trial.suggest_int('max_depth', 3, 10, step=1),
        'learning_rate'   : trial.suggest_float('learning_rate', 0.01, 1),
        'lags'            : trial.suggest_categorical('lags', lags_grid)
    } 
    return search_space

bayesian_search_multiseries_benchmark(
    data                    = data,
    forecaster_to_benchmark = forecaster,
    search_space            = search_space,
    end_train               = end_train,
    end_validation          = end_validation,
    levels                  = levels,
    exog_features           = exog_features,
    steps                   = 36,
    metric                  = 'mean_absolute_error'
)
results_grid_search.loc["items_sales", :] = [
    type(forecaster).__name__,
    time_1,
    time_2,
    metric_1,
    metric_2,
]

  0%|          | 0/20 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48] 
  Parameters: {'n_estimators': 47, 'max_depth': 10, 'learning_rate': 0.10582782597735779}
  Backtesting metric: 1.9340570479344343
  Levels: ['item_1', 'item_2', 'item_3']



  0%|          | 0/20 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72] 
  Parameters: {'n_estimators': 35, 'max_depth': 3, 'learning_rate': 0.3241126270021177}
  Backtesting metric: 1.586531587692841
  Levels: ['item_1', 'item_2', 'item_3']

Benchmark results
-----------------
Execution time backtesting   : 5.048365592956543
Execution time one step ahead: 1.8876800537109375
Same lags  : False
Same params: False

Method: backtesting
    lags   : [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48]
    params : {'n_estimators': 47, 'max_depth': 10, 'learning_rate': 0.10582782597735779}
    metrics: 1.2509853110935267

Method: one step ahead
    lags   : 

## ForecasterAutoregMultiVariate

In [55]:
# Data download
# ==============================================================================
data = fetch_dataset(name="items_sales")
end_train = '2014-05-15 23:59:00'
end_validation = '2014-07-15 23:59:00'
data_train = data.loc[:end_train, :].copy()
data_test  = data.loc[end_train:, :].copy()
levels = ['item_1', 'item_2', 'item_3']
exog_features = None 

items_sales
-----------
Simulated time series for the sales of 3 different items.
Simulated data.
Shape of the dataset: (1097, 3)


In [56]:
forecaster = ForecasterAutoregMultiVariate(
                 regressor          = LGBMRegressor(random_state=123, verbose=-1),
                 lags               = 24,
                 steps              = 5,
                 level              = 'item_1',
                 transformer_series = None,
                 transformer_exog   = None,
                 weight_func        = None,
                 fit_kwargs         = None,
                 forecaster_id      = None
             )

lags_grid ={
    '24 lags': 24,
    '48 lags': 48
}

param_grid = {
    'n_estimators': [10, 20],
    'max_depth': [3, 7]
}

grid_search_multiseries_benchmark(
    data                    = data,
    forecaster_to_benchmark = forecaster,
    lags_grid               = lags_grid,
    param_grid              = param_grid,
    end_train               = end_train,
    end_validation          = end_validation,
    levels                  = levels,
    exog_features           = exog_features,
    steps                   = 5
)

lags grid:   0%|          | 0/2 [00:00<?, ?it/s]

params grid:   0%|          | 0/4 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48] 
  Parameters: {'max_depth': 7, 'n_estimators': 20}
  Backtesting metric: 0.7515349140955219
  Levels: ['item_1']



lags grid:   0%|          | 0/2 [00:00<?, ?it/s]

params grid:   0%|          | 0/4 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24] 
  Parameters: {'max_depth': 7, 'n_estimators': 20}
  Backtesting metric: 0.8146025385778681
  Levels: ['item_1']

Benchmark results
-----------------
Execution time backtesting   : 3.104400873184204
Execution time one step ahead: 2.043469190597534
Same lags  : False
Same params: True

Method: backtesting
    lags   : [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48]
    params : {'max_depth': 7, 'n_estimators': 20}
    metrics: 1.0409310906937037

Method: one step ahead
    lags   : [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]
    params : {'max_depth': 7, 'n_estimators': 20}
    metrics: 1.0481118869071948


# Unit test

## _evaluate_grid_hyperparameters vs _evaluate_grid_hyperparameters_one_step_ahead

In [26]:
from skforecast.model_selection.model_selection import _evaluate_grid_hyperparameters
from skforecast.model_selection.model_selection import _evaluate_grid_hyperparameters_one_step_ahead
from sklearn.model_selection import ParameterGrid

In [27]:
# Data
# ==============================================================================
data = fetch_dataset(
    name="h2o",
    raw=True,
    kwargs_read_csv={"names": ["y", "datetime"], "header": 0},
    verbose=False,
)
data["datetime"] = pd.to_datetime(data["datetime"], format="%Y-%m-%d")
data = data.set_index("datetime")
data = data.asfreq("MS")
data = data[["y"]]
data = data.sort_index()
end_train = "2001-01-01 23:59:00"
end_validation = "2006-01-01 23:59:00"
target = "y"
exog_features = None

In [28]:
forecaster = ForecasterAutoreg(
                regressor = LGBMRegressor(random_state=123, verbose=-1),
                lags      = 10
            )
lags_grid = [3, 10, [1, 2, 3, 20]]
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [2, 3, 5]
}
param_grid = list(ParameterGrid(param_grid))

results_backtesting = _evaluate_grid_hyperparameters(
    forecaster         = forecaster,
    y                  = data.loc[:end_validation, target],
    exog               = data.loc[:end_validation, exog_features] if exog_features else None,
    param_grid         = param_grid,
    lags_grid          = lags_grid,
    steps              = 1,
    refit              = False,
    metric             = ['mean_squared_error', 'mean_absolute_error'],
    initial_train_size = len(data.loc[:end_train]),
    fixed_train_size   = False,
    return_best        = False,
    n_jobs             = 'auto',
    verbose            = False,
    show_progress      = False
)

results_one_step_ahead = _evaluate_grid_hyperparameters_one_step_ahead(
    forecaster         = forecaster,
    y                  = data.loc[:end_validation, target],
    exog               = data.loc[:end_validation, exog_features] if exog_features else None,
    param_grid         = param_grid,
    lags_grid          = lags_grid,
    metric             = ['mean_squared_error', 'mean_absolute_error'],
    initial_train_size = len(data.loc[:end_train]),
    return_best        = False,
    verbose            = False,
    show_progress      = False
)

assert results_backtesting.equals(results_one_step_ahead)

/home/ubuntu/varios/skforecast/skforecast/model_selection/model_selection.py:1435: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


## _calculate_metrics_multiseries vs _calculate_metrics_multiseries_one_step_ahead

In [29]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from skforecast.model_selection_multiseries.model_selection_multiseries import _calculate_metrics_multiseries_one_step_ahead
from skforecast.model_selection_multiseries.model_selection_multiseries import _calculate_metrics_multiseries
from skforecast.model_selection.model_selection import _create_backtesting_folds
from skforecast.metrics import add_y_train_argument
from skforecast.metrics import mean_absolute_scaled_error

In [30]:
# Data download
# ==============================================================================
data = fetch_dataset(name="items_sales")
end_train = '2014-07-15 23:59:00'
data_train = data.loc[:end_train, :].copy()
data_test  = data.loc[end_train:, :].copy()

items_sales
-----------
Simulated time series for the sales of 3 different items.
Simulated data.
Shape of the dataset: (1097, 3)


In [31]:
forecaster = ForecasterAutoregMultiSeries(
                 regressor          = LGBMRegressor(random_state=123, verbose=-1),
                 lags               = 24,
                 encoding           = 'ordinal',
                 transformer_series = None,
                 transformer_exog   = None,
                 weight_func        = None,
                 series_weights     = None,
                 differentiation    = None,
                 dropna_from_series = False,
                 fit_kwargs         = None,
                 forecaster_id      = None
             )

X, y = forecaster.create_train_X_y(series=data)
X_train = X.loc[X.index <= end_train, :]
y_train = y.loc[y.index <= end_train]
X_test = X.loc[X.index > end_train, :]
y_test = y.loc[y.index > end_train]

forecaster.regressor.fit(X_train, y_train)
pred = forecaster.regressor.predict(X_test)

In [32]:
# Test results _calculate_metrics_multiseries_one_step_ahead and _calculate_metrics_multiseries
# ==============================================================================
# Metrics should be equal when using step=1 in backtesting

# Metrics with _calculate_metrics_multiseries_one_step_ahead
metrics = [mean_absolute_error, mean_absolute_percentage_error, mean_absolute_scaled_error]
metrics = [add_y_train_argument(metric) for metric in metrics]

if forecaster.encoding in ['ordinal', 'ordinal_category']:
    X_train_encoding = forecaster.encoder.inverse_transform(X_train[['_level_skforecast']]).ravel()
    X_test_encoding = forecaster.encoder.inverse_transform(X_test[['_level_skforecast']]).ravel()
elif forecaster.encoding == 'onehot':
    X_train_encoding = forecaster.encoder.inverse_transform(
                            X_train.loc[:, forecaster.encoding_mapping.keys()]
                        ).ravel()
    X_test_encoding = forecaster.encoder.inverse_transform(
                            X_test.loc[:, forecaster.encoding_mapping.keys()]
                        ).ravel()

metrics_one_step_ahead = _calculate_metrics_multiseries_one_step_ahead(
    y_true = y_test.to_numpy(),
    y_true_index = y_test.index,
    y_pred = pred,
    y_pred_encoding = X_test_encoding,
    y_train = y_train.to_numpy(),
    y_train_index = y_train.index,
    y_train_encoding = X_train_encoding,
    levels = ['item_1', 'item_2', 'item_3'],
    metrics = metrics,
    add_aggregated_metric = True
)

display(metrics_one_step_ahead)

# Metrics with _calculate_metrics_multiseries
folds = _create_backtesting_folds(
    data = data,
    window_size = 24,
    initial_train_size = len(data_train),
    test_size = 1,
    externally_fitted = False,
    refit = False,
    fixed_train_size = True,
    gap = 0,
    skip_folds = None,
    allow_incomplete_fold = True,
    return_all_indexes = False,
    differentiation = None,
    verbose = False
)
_, predictions = backtesting_forecaster_multiseries(
    series=data,
    forecaster=forecaster,
    steps=1,
    metric=metrics,
    initial_train_size = len(data_train),
    refit=False,
    add_aggregated_metric=True,
    show_progress=False
)
metrics_backtesting = _calculate_metrics_multiseries(
    series = data,
    predictions= predictions,
    folds= folds,
    span_index= data.index,
    window_size = 24,
    metrics= metrics,
    levels= ['item_1', 'item_2', 'item_3'],
    add_aggregated_metric = True
)

display(metrics_backtesting)

assert metrics_backtesting.equals(metrics_one_step_ahead)

,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,item_1,0.820748,0.040761,0.537711
1,item_2,2.330964,0.152464,0.989540
2,item_3,3.093253,0.207347,0.835301
3,average,2.081655,0.133524,0.787517
4,weighted_average,2.081655,0.133524,0.787517
5,pooling,2.081655,0.133524,0.823316


,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,item_1,0.820748,0.040761,0.537711
1,item_2,2.330964,0.152464,0.989540
2,item_3,3.093253,0.207347,0.835301
3,average,2.081655,0.133524,0.787517
4,weighted_average,2.081655,0.133524,0.787517
5,pooling,2.081655,0.133524,0.823316


## _evaluate_grid_hyperparameters_multiseries vs _evaluate_grid_hyperparameters_multiseries_one_step_ahead 

In [33]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from skforecast.model_selection_multiseries.model_selection_multiseries import _evaluate_grid_hyperparameters_multiseries
from skforecast.model_selection_multiseries.model_selection_multiseries import _evaluate_grid_hyperparameters_multiseries_one_step_ahead
from skforecast.metrics import mean_absolute_scaled_error

In [34]:
# Data download
# ==============================================================================
data = fetch_dataset(name="items_sales")
end_train = '2014-07-15 23:59:00'
data_train = data.loc[:end_train, :].copy()
data_test  = data.loc[end_train:, :].copy()
exog_features = None

items_sales
-----------
Simulated time series for the sales of 3 different items.
Simulated data.
Shape of the dataset: (1097, 3)


In [35]:
forecaster = ForecasterAutoregMultiSeries(
    regressor          = LGBMRegressor(random_state=123, verbose=-1),
    lags               = 24,
    encoding           = 'ordinal',
    transformer_series = None,
    transformer_exog   = None,
    weight_func        = None,
    series_weights     = None,
    differentiation    = None,
    dropna_from_series = False,
    fit_kwargs         = None,
    forecaster_id      = None
)
lags_grid = [3, 10]
param_grid = {
    'n_estimators': [5, 10],
    'max_depth': [2, 3]
}
param_grid = list(ParameterGrid(param_grid))

results_backtesting = _evaluate_grid_hyperparameters_multiseries(
                            forecaster            = forecaster,
                            series                = data,
                            param_grid            = param_grid,
                            steps                 = 1,
                            metric                = metrics,
                            aggregate_metric      = ['weighted_average', 'average', 'pooling'],
                            initial_train_size    = len(data_train),
                            fixed_train_size      = True,
                            gap                   = 0,
                            skip_folds            = None,
                            allow_incomplete_fold = True,
                            levels                = None,
                            exog                  = data.loc[:end_validation, exog_features] if exog_features else None,
                            lags_grid             = lags_grid,
                            refit                 = False,
                            n_jobs                = 'auto',
                            return_best           = False,
                            verbose               = False,
                            show_progress         = False,
                            suppress_warnings     = False
                        )

results_one_step_ahead = _evaluate_grid_hyperparameters_multiseries_one_step_ahead(
                            forecaster            = forecaster,
                            series                = data,
                            param_grid            = param_grid,
                            metric                = metrics,
                            initial_train_size    = len(data_train),
                            exog                  = data.loc[:data_train, exog_features] if exog_features else None,
                            lags_grid             = lags_grid,
                            return_best           = False,
                            verbose               = False,
                            show_progress         = False
                        )

assert results_backtesting.equals(results_one_step_ahead)


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:2029: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


In [62]:
forecaster = ForecasterAutoregMultiVariate(
                 regressor          = LGBMRegressor(random_state=123, verbose=-1),
                 lags               = 24,
                 steps              = 2,
                 level              = 'item_1',
                 transformer_series = None,
                 transformer_exog   = None,
                 weight_func        = None,
                 fit_kwargs         = None,
                 forecaster_id      = None
             )

X, y = forecaster.create_train_X_y(series=data)
train_size = len(data_train) - forecaster.window_size_diff
X_train = X.iloc[:train_size, :]
X_test = X.iloc[train_size:, :]
y_train = {k: v.iloc[:train_size] for k, v in y.items()}
y_test  = {k: v.iloc[train_size:] for k, v in y.items()}

step=1

X_train_step, y_train_step = forecaster.filter_train_X_y_for_step(
                                                    step    = step,
                                                    X_train = X_train,
                                                    y_train = y_train
                                                  )
X_test_step, y_test_step = forecaster.filter_train_X_y_for_step(
                                step    = step,  
                                X_train = X_test,
                                y_train = y_test
                            )

In [50]:
forecaster = ForecasterAutoregMultiSeries(
                regressor          = LGBMRegressor(random_state=123, verbose=-1),
                lags               = 24,
                encoding           = 'ordinal',
                transformer_series = None,
                transformer_exog   = None,
                weight_func        = None,
                fit_kwargs         = None,
                forecaster_id      = None
            )

X, y, *_ = forecaster._create_train_X_y(series=data)


In [45]:
X

,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,...,lag_16,lag_17,lag_18,lag_19,lag_20,lag_21,lag_22,lag_23,lag_24,_level_skforecast
date,,,,,,,,,,,,,,,,,,,,,
2012-01-25,28.018830,23.981037,20.794986,22.503533,24.018768,24.772249,29.245869,26.636444,20.228468,18.976196,...,20.006161,20.069327,20.533871,21.106643,21.379238,25.895533,27.549099,22.777826,8.253175,0
2012-01-26,28.747482,28.018830,23.981037,20.794986,22.503533,24.018768,24.772249,29.245869,26.636444,20.228468,...,21.620184,20.006161,20.069327,20.533871,21.106643,21.379238,25.895533,27.549099,22.777826,0
2012-01-27,23.908368,28.747482,28.018830,23.981037,20.794986,22.503533,24.018768,24.772249,29.245869,26.636444,...,21.717691,21.620184,20.006161,20.069327,20.533871,21.106643,21.379238,25.895533,27.549099,0
2012-01-28,21.423930,23.908368,28.747482,28.018830,23.981037,20.794986,22.503533,24.018768,24.772249,29.245869,...,21.751748,21.717691,21.620184,20.006161,20.069327,20.533871,21.106643,21.379238,25.895533,0
2012-01-29,24.786455,21.423930,23.908368,28.747482,28.018830,23.981037,20.794986,22.503533,24.018768,24.772249,...,21.758617,21.751748,21.717691,21.620184,20.006161,20.069327,20.533871,21.106643,21.379238,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-12-28,21.738537,19.434599,15.148365,22.075515,25.114921,25.982370,18.457338,14.950277,20.415604,20.758476,...,20.488914,18.975586,21.678973,17.274616,13.292155,19.417643,16.099237,19.913343,23.642604,2
2014-12-29,20.586030,21.738537,19.434599,15.148365,22.075515,25.114921,25.982370,18.457338,14.950277,20.415604,...,19.785106,20.488914,18.975586,21.678973,17.274616,13.292155,19.417643,16.099237,19.913343,2
2014-12-30,28.127390,20.586030,21.738537,19.434599,15.148365,22.075515,25.114921,25.982370,18.457338,14.950277,...,30.684650,19.785106,20.488914,18.975586,21.678973,17.274616,13.292155,19.417643,16.099237,2


In [71]:
forecaster = ForecasterAutoregMultiSeries(
                 regressor          = LGBMRegressor(random_state=123, verbose=-1),
                 lags               = 24,
                 encoding           = "ordinal",
                 transformer_series = None,
                 transformer_exog   = None,
                 weight_func        = None,
                 series_weights     = None,
                 differentiation    = None,
                 dropna_from_series = False,
                 fit_kwargs         = None,
                 forecaster_id      = None
             )

lags_grid = [48, 72]
def search_space(trial):
    search_space  = {
        'n_estimators'    : trial.suggest_int('n_estimators', 10, 50),
        'max_depth'       : trial.suggest_int('max_depth', 3, 10, step=1),
        'learning_rate'   : trial.suggest_float('learning_rate', 0.01, 1),
        'lags'            : trial.suggest_categorical('lags', lags_grid)
    } 
    return search_space

results, trial = bayesian_search_forecaster_multiseries(
    series                    = data,
    forecaster = forecaster,
    method='one_step_ahead',
    initial_train_size= len(data_train),
    search_space               = search_space,
    levels                  = levels,
    metric='mean_absolute_error',
    steps                   = 36,
    verbose=False
)
results

  0%|          | 0/10 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72] 
  Parameters: {'n_estimators': 40, 'max_depth': 4, 'learning_rate': 0.1836972385860176}
  Backtesting metric: 1.9355197828331292
  Levels: ['item_1', 'item_2', 'item_3']



,levels,lags,params,mean_absolute_error__weighted_average,mean_absolute_error__average,mean_absolute_error__pooling,n_estimators,max_depth,learning_rate
0,"[item_1, item_2, item_3]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'n_estimators': 40, 'max_depth': 4, 'learning...",1.935520,1.935520,1.935520,40.0,4.0,0.183697
1,"[item_1, item_2, item_3]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'n_estimators': 35, 'max_depth': 3, 'learning...",1.949453,1.949453,1.949453,35.0,3.0,0.324113
2,"[item_1, item_2, item_3]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'n_estimators': 38, 'max_depth': 5, 'learning...",1.997867,1.997867,1.997867,38.0,5.0,0.234583
3,"[item_1, item_2, item_3]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'n_estimators': 23, 'max_depth': 5, 'learning...",2.008125,2.008125,2.008125,23.0,5.0,0.235981
4,"[item_1, item_2, item_3]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'n_estimators': 13, 'max_depth': 6, 'learning...",2.110749,2.110749,2.110749,13.0,6.0,0.436554
5,"[item_1, item_2, item_3]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'n_estimators': 24, 'max_depth': 8, 'learning...",2.157664,2.157664,2.157664,24.0,8.0,0.444187
6,"[item_1, item_2, item_3]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'n_estimators': 27, 'max_depth': 10, 'learnin...",2.443927,2.443927,2.443927,27.0,10.0,0.687981
7,"[item_1, item_2, item_3]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'n_estimators': 22, 'max_depth': 6, 'learning...",2.569656,2.569656,2.569656,22.0,6.0,0.894455
8,"[item_1, item_2, item_3]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'n_estimators': 36, 'max_depth': 9, 'learning...",2.621470,2.621470,2.621470,36.0,9.0,0.727211
9,"[item_1, item_2, item_3]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'n_estimators': 20, 'max_depth': 6, 'learning...",2.746263,2.746263,2.746263,20.0,6.0,0.985704
